### Author: Rodolfo Elenes

Date Created: 8/14/2025

Change log:
8/14/2025 - Initialized

##### Imports

In [1]:
import pandas as pd
import numpy as np
import re
import requests
import duckdb
import time
from pathlib import Path
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [2]:
%run ./common_utils.ipynb

Importing following functions: save_df(), construct_df_roster(), construct_df_teams(), concatenate_all_files()


##### Notebook Functions

In [3]:
def construct_wiki_tbl(con_memory):
#   Function name: construct_wiki_tbl
#   Description: Create the base of the Wikipedia table
#   Parameters: con_memory
#        con_memory(ducbdb object): used to carry duckdb queries
#   Return values: wiki
#        wiki(pandas dataframe): The dataframe with all suspension data from Wikipedia

    url = "https://en.wikipedia.org/wiki/List_of_suspensions_in_the_NFL"
    print("Attempting to construct Wikipedia table from:", url)
    
    # Scrape data from Wikipedia
    columns = ["Date suspended", "Suspension length", "Name", "Position", "Team at the time of suspension"]
    df = pd.DataFrame(columns = columns)
    for i in [0, 1]: # Gather Player suspensions + Suspensions for violating the substance policies
        time.sleep(6) # Respect website scraping policies
        df_temp = scrape_web_src(url, i)
        df_temp = df_temp[columns]
        df = pd.concat([df, df_temp])

    df['Date'] = pd.to_datetime(df['Date suspended'], format="%B %d, %Y", errors='coerce')
    df['Date'] = np.where(df.Date.isnull(), pd.to_datetime(df['Date suspended'], format="%B %Y", errors='coerce'), df.Date)

    # Column renaming
    edit_df_cols = df.columns.tolist()
    final_columns = ['Date', 'Susp_len', 'Player', 'Position', 'Team', 'Drop']
    for i in range(df.shape[1]):
        edit_df_cols[i] = final_columns[i]
    df.columns = edit_df_cols

    # Dataframe transformations
    df['Date'] = df['Drop']
    df = df.drop('Drop', axis=1)
    df['Position'] = np.where(df['Position'].isin(['FB', 'HB']), 'RB', df.Position) # THINK ABOUT: u want to convert positions to RB
    df = df[(df['Date'] >= '1990') & (df['Position'] == 'RB') & (~df.Susp_len.str.contains('overturned', case=False))]

    # Hard code fixes
    df['Susp_len'] = np.where(df.Susp_len == 'Indefinite (reinstated in Feb. 2015)[116]', 'Entire 2014 season', df.Susp_len)
    df['Susp_len'] = np.where(df.Susp_len == 'Indefinite[d] (reinstated four days later)[133]', '1 games', df.Susp_len)
    df['Susp_len'] = np.where(df.Susp_len == 'Indefinite (reinstated in Aug. 2012)[311]', 'Indefinite', df.Susp_len)
    df['Susp_len'] = np.where(df.Susp_len == 'Indefinite (reinstated in Dec. 2016)[532]', 'Entire 2016 season', df.Susp_len)
    df['Susp_len'] = np.where(df.Susp_len == 'Indefinite', "Entire " + df['Date'].dt.year.astype(str) + " season", df.Susp_len)
    df['Susp_len'] = np.where(df.Susp_len == '3 games (later reduced to 2 games)[476]', '2 games', df.Susp_len)

    df = df.sort_values('Date').reset_index(drop=True)

    # Get rows with "Entire" in Susp_len
    df2 = df.query("Susp_len.str.contains('entire', case=False)")
    susp_list1 = df2.Susp_len.unique().tolist()

    # Get rows with only number of game suspensions defined
    df3 = df[(~df['Susp_len'].isin(susp_list1))]
    susp_list2 = df3.Susp_len.tolist()
    fltrd_susp_list2 = []
    for i in susp_list2: # Remove inconsistent games/games[d] phrasing
        fltrd_susp_list2.append(i.split(" ")[0])    
    df3['Susp_len'] = fltrd_susp_list2
    df3['Susp_len'] = df3.Susp_len + " games" # Added back games for consistency

    wiki = pd.concat([df2, df3])
    
    # Attach Season and Week information to the suspended players
    df_dates = pd.read_csv("../tables/nfl_dates_xref.csv")
    wiki = con_memory.execute("SELECT * FROM wiki JOIN df_dates ON df_dates.Date = wiki.Date").fetchdf()
    wiki['Date'] = pd.to_datetime(wiki.Date, format='%Y-%m-%d')
    print("Wikipedia table constructed.")
    
    return wiki

In [4]:
def construct_pst_tbl(con_memory):
#   Function name: construct_pst_tbl
#   Description: Create the base of the PST table
#   Parameters: con_memory
#        con_memory(ducbdb object): used to carry duckdb queries
#   Return values: df
#        df(pandas dataframe): The dataframe with all suspension data from prosportstransactions.com

    src_tbl = "../src/nfl_suspensions_by_pst.csv"
    print("Attempting to construct Prosportstransactions.com table from:", src_tbl)
    df_roster = construct_df_roster(con_memory)
    df_roster['short_name'] = df_roster.Team.str.split(" ").str[-1]
    df_roster['short_name'] = np.where(df_roster.Team == 'Washington Football Team', 'Washington', df_roster.short_name)
    con_memory.register('roster', df_roster)
    
    # Construct suspensions data from Prosportstransactions.com and join with rosters table to bring identifiers to suspended players
    df = pd.read_csv(src_tbl)
    df = df[['Date', 'Team', 'Acquired', 'Relinquished', 'Notes']]
    df['Acquired'] = np.where(df.Acquired == '#VALUE!', np.nan, df.Acquired)
    df['Relinquished'] = np.where(df.Relinquished == '#VALUE!', np.nan, df.Relinquished)
    df['Team'] = np.where(df.Team.isnull(), 'Free Agent', df.Team)
    df['Date'] = pd.to_datetime(df.Date)
    df['name'] = np.where(df.Acquired.isnull() == False, df.Acquired, np.nan)
    df['name'] = np.where(df.Relinquished.isnull() == False, df.Relinquished, np.nan)
    df = con_memory.execute("SELECT * FROM roster JOIN df ON roster.Player = df.name AND roster.short_name = df.Team").fetchdf()
    con_memory.register('pst', df)

    # Attach Season and Week information to the suspended players and filter out fines entries
    df_dates = pd.read_csv("../tables/nfl_dates_xref.csv")
    df = con_memory.execute("""SELECT Season, Week, Date, Team, Player, Notes AS Susp_len FROM 
                               (SELECT * FROM df_dates JOIN pst ON df_dates.Date = pst.Date) 
                               WHERE Notes NOT ILIKE '%fine%' 
                               ORDER BY Date""").fetchdf()

    # Extract suspension lengths
    filter_list = df.Susp_len.unique()
    pattern = r"\b\d+\s+games?\b"
    matches = []
    for text in filter_list:
        found = re.findall(pattern, text.lower())
        if found:
            matches.extend(found)
            df['Susp_len'] = np.where(df.Susp_len == text, found[0], df.Susp_len)
    df['Date'] = pd.to_datetime(df.Date, format='%Y-%m-%d')
    df['Susp_len'] = np.where(df['Susp_len'].str.contains('indef'), "Entire " + df['Date'].dt.year.astype(str) + " season", df.Susp_len)
    df = df[(df['Susp_len'] != 'suspended from practice squad')] # Not including practice squad suspensions
    print("PST table constructed.")
    
    return df

##### Other Functions

In [5]:
def scrape_web_src(url, tbl=0):
#   Function name: scrape_web_src
#   Description: This function is used to provide a stronger GET request to scrape web data
#   Parameters: url, tbl
#        url(str): The target URL
#        tbl(int): The target table from the scraped web source
#   Return values: df
#        tables(pandas dataframe): The target table from the URL

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        tables = pd.read_html(response.text)
        tables = tables[tbl]
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        
    return tables

In [6]:
def final_tbl_corrections(df):
#   Function name: final_tbl_corrections
#   Description: Make manual edits to the final table to ensure a smoother process for later scripts
#   Parameters: df
#        df(pandas dataframe): Final table to be edited
#   Return values: df
#        df(pandas dataframe): Final table with edits
    
    # Ezekiel Elliot 2017 suspension, refined to one entry for the final table:
    drop_indx = df[(df.Player == 'Ezekiel Elliott') & (df.Date == '2017-08-11')].index[0]
    df = df.drop(drop_indx)
    print("\nManually removed Ezekiel Elliott[2017-08-11] 6 game suspension entry \nas the suspension was reinstated later on for weeks 10-15 of 2017 season.\n")
    
    return df

##### Main

In [7]:
def main():
#   Function name: main
#   Description: The entry function of the notebook

    con_memory = duckdb.connect(database=':memory:')
    wiki = construct_wiki_tbl(con_memory)
    suspensions_pst = construct_pst_tbl(con_memory)
    
    # Union the Wikipedia and PST data to get a unique table with all suspension entries publicly available
    df = con_memory.execute("""SELECT * FROM 
                               (SELECT * FROM suspensions_pst 
                               UNION 
                               SELECT Season, Week, Date, Team, Player, Susp_len FROM wiki)
                               ORDER BY Date, Player""").fetchdf()
    df = final_tbl_corrections(df)
    save_df(df, '../tables', 'susp_weeks_xref.csv')
    display(df)
    con_memory.close()
    
main()

Attempting to construct Wikipedia table from: https://en.wikipedia.org/wiki/List_of_suspensions_in_the_NFL
Wikipedia table constructed.
Attempting to construct Prosportstransactions.com table from: ../src/nfl_suspensions_by_pst.csv
PST table constructed.

Manually removed Ezekiel Elliott[2017-08-11] 6 game suspension entry 
as the suspension was reinstated later on for weeks 10-15 of 2017 season.

Saving susp_weeks_xref.csv at ../tables/susp_weeks_xref.csv
Successfully saved susp_weeks_xref.csv!


,Season,Week,Date,Team,Player,Susp_len
0,1990,0,1990-08-30,Indianapolis Colts,Eric Dickerson,6 games
1,1990,4,1990-10-05,Green Bay Packers,Herman Fontenot,1 games
2,1991,9,1991-10-31,Pittsburgh Steelers,Tim Worley,6 games
3,1991,10,1991-11-08,Indianapolis Colts,Eric Dickerson,4 games
4,1992,0,1992-04-30,Pittsburgh Steelers,Tim Worley,Entire 1992 season
...,...,...,...,...,...,...
88,2019,0,2019-07-16,Minnesota Vikings,Roc Thomas,3 games
89,2019,9,2019-11-04,Miami Dolphins,Mark Walton,4 games
90,2020,4,2020-10-06,Tampa Bay Buccaneers,Kenjon Barner,4 games
91,2022,4,2022-10-03,Kansas City Chiefs,Jerrion Ealy,6 games
